In [2]:
from datetime import datetime
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = "seaborn"

In [45]:
df_uk_death = pd.read_excel('../data/external/uk_covid_death.xlsx', sheet_name='Deaths', header=3, nrows=95, engine='openpyxl')
df_uk_death.drop(columns='Unnamed: 10', inplace=True)
df_uk_death.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
df_uk_death['Under 14'] = df_uk_death['Under 1 year'] + df_uk_death['1 to 14']
df_uk_death.drop(columns=['Under 1 year', '1 to 14'], inplace=True)
df_uk_death.set_index('date', inplace=True)
df_uk_death.columns = [df_uk_death.columns[-1]] + [c.strip() for c in df_uk_death.columns[:-1]]

df_china_age_group = pd.read_csv('../data/processed/china_gender_age_aggregated.csv')
df_china_age_group.rename(columns={'person': '%population'}, inplace=True)
df_uk_population = pd.read_csv('../data/processed/uk_population_2022_aggregated.csv')

In [46]:
df_list = []
for i in range(df_uk_death.shape[0]):
    df = df_uk_death.iloc[i:].sum().to_frame()
    df_list.append(df)
df_uk_death_agg = pd.concat(df_list, axis=1).T.set_index(df_uk_death.index)
uk_death_rate = df_uk_death_agg.div(df_uk_population[['age', 'population']].set_index('age').T.iloc[0] * 1000) 

In [47]:
china_death_scenarios = uk_death_rate.mul(df_china_age_group.set_index('age').T.iloc[0]) * 1412
china_death_scenarios['total'] = china_death_scenarios.apply(sum, axis=1) * 1000 * 1000
china_death_scenarios.reset_index(inplace=True)

In [37]:
fig = px.line(
    china_death_scenarios, x='date', y='total',
    labels={'date': 'COVID-zero policy end date', 'total': 'Fatality estimate'},
)

for month_start in ['2021-02-12', '2021-06-04', '2021-12-03', '2022-01-07', '2022-06-03', '2022-12-02']:
    start_date = datetime.strptime(month_start, '%Y-%m-%d').timestamp() * 1000
    death = china_death_scenarios.query("date==@month_start").total.values[0].astype(int)
    fig.add_annotation(
        x=start_date,
        y=death,
        text=f'{death:,}',
        showarrow=False,
        yshift=15
    )
    fig.add_vline(
        x=start_date,
        line=dict(width=1.5, dash='dash'),
    )

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(
    font_size=18,
    margin=dict(
        l=50,
        r=20,
        t=20,
        b=50,
        pad=0
    ),
    width=1200,
    height=600
)

fig.show()

In [38]:
estimate_total = pio.write_json(fig, '../data/plotly_json/estimate_total.json')

In [25]:
timeline_estimate = pd.melt(china_death_scenarios, 
        id_vars=['date'],
        value_vars=['Under 14', '15 to 24', '25 to 44', '45 to 54', '55 to 64', '65 to 74', '75 to 84', '85+'],
        var_name='age',
        value_name='deaths'
       )
timeline_estimate.date = timeline_estimate.date.dt.strftime('%Y-%m-%d')
timeline_estimate.deaths = (timeline_estimate.deaths*1000*1000).astype(int)

In [53]:
fig = px.bar(
    timeline_estimate, orientation='h',
    y="age", x="deaths", color='age', text='deaths',
    animation_frame="date", animation_group="age",
    labels={'age': 'Age group', 'deaths': 'Fatality estimate'},
    range_x=(0, 400000)
)

time = china_death_scenarios.reset_index().date.dt.strftime('%Y-%m-%d').values
estimate = china_death_scenarios.total.values.astype(int)
for i, frame in enumerate(fig.frames):
    frame.layout.title = f'COVID-zero policy end date: {time[i]} & Total fatality estimate: {estimate[i]:,}'
    
for step in fig.layout.sliders[0].steps:
    step["args"][1]["frame"]["redraw"] = True

fig.update_traces(textposition='inside', texttemplate='%{text:,}')
fig.update_layout(
    showlegend=False, 
    title_x=0.5,
    font_size=18,
    margin=dict(
        l=50,
        r=20,
        t=50,
        b=50,
        pad=0
    ),
    width=1200,
    height=600
)
fig.show()

In [54]:
estimate_age_group = pio.write_json(fig, '../data/plotly_json/estimate_age_group.json')

In [15]:
import io
import PIL

fig.layout.pop('sliders')
fig.layout.pop('updatemenus')
time = china_death_scenarios.date.dt.strftime('%Y-%m-%d').values
estimate = (china_death_scenarios.total.values).astype(int)
    
frames = []
for i, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    fig.layout.title = f'COVID-zero policy end date: {time[i]} & Total fatality estimate: {estimate[i]:,}'
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        '../reports/figures/scenarios.gif',
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=200,
        loop=0,
    )

In [52]:
data_store = '../data/store.h5'
uk_death_rate.to_hdf(data_store, 'uk_death_rate_weekly')
df_china_age_group.to_hdf(data_store, 'china_age_group')